<a href="https://colab.research.google.com/github/bbarber314/MATH527_MLForFinance/blob/main/Homework3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Modify the CNN 1D time series notebook to predict high frequency
mid-prices with a single hidden layer CNN, using the data HFT.csv.
Then complete the following tasks

1.   Confirm that the data is stationary by applying the augmented Dickey-Fuller test
2.   Estimate the partial autocorrelation and determine the optimum lag at the 99% confidence level.
3.   Evaluate the MSE in-sample and out-of-sample using 4 filters.
What do you conclude about the level of over-fitting as you vary
the number of filters?
4.   Apply L1 regularization to reduce the variance.
5.   Determine whether the model error is white noise or is autocorrelated by applying the Ljung-Box test.

Hint: You should review the HFT RNN notebook before you begin this
exercise.
___

Prep work

In [2]:
from google.colab import drive # import drive from google colab; mount my google drive to connect to the data file

ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)

drive.mount(ROOT)           # we mount the google drive at /content/drive%pwd

/content/drive
Mounted at /content/drive


In [4]:
%ls

drive/  sample_data/


In [2]:
%pwd

'/content'

In [6]:
%cd drive/MyDrive/Colab\ Notebooks/data
# jump to the data folder

/content/drive/MyDrive/Colab Notebooks/data


In [63]:
#load libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import tensorflow as tf

from keras.layers import Conv1D, Dense, MaxPooling1D, Flatten
from keras.models import Sequential

from sklearn.metrics import mean_squared_error

In [8]:
df = pd.read_csv('HFT.csv') #read the CSV (why >30 MB?!?!?!?)

In [32]:
use_features = ['feature_3'] # continuous input, variable of interest (could also use feature 1 or 2)
target = ['feature_3'] # continuous output
n_steps_ahead = 10 # forecasting horizon
dataSet = df['feature_3'][:20000] # using the full data set crashed colab when performing the augmented Dickey Fuller. 200000 yielded 2+min/epoch for 1 kernel CNN

___

1.   Check for stationarity with augmented Dickey Fuller


In [33]:
adf, p, usedlag, nobs, cvs, aic = sm.tsa.stattools.adfuller(dataSet)
adf_results_string = 'ADF: {}\np-value: {},\nN: {}, \ncritical values: {}'
print(adf_results_string.format(adf, p, nobs, cvs))

ADF: -5.314335374054773
p-value: 5.113419156150479e-06,
N: 19964, 
critical values: {'1%': -3.4306775967247427, '5%': -2.8616847862243144, '10%': -2.5668470657535196}


The p-value is less than 0.01, so at the 99% level, we can reject the null hypothesis in favor of the alternative: there are no roots within the unit circle.
___
2.   Estimate PACF and decide the number of lags based on a 99% confidence estimate.

In [34]:
pacf = sm.tsa.stattools.pacf(df[use_features], nlags=50)

In [37]:
T = len(df[use_features])
sig_test = lambda tau_h: np.abs(tau_h) > 2.576/np.sqrt(T) # 2.58 is approximately the 99% Z-score. I'm going to use 2.576 just to be special
for i in range(len(pacf)):
    if sig_test(pacf[i]) == False:
        n_steps = i - 1
        print('n_steps set to', n_steps)
        break

n_steps set to 23


Using 68% CI yields 30 steps, 99.73% CI yields 11 steps, and finally at the 99% CI, we use 23 steps.
___
3) Estimate the MSE in-sample and out of sample for four different sets of filters. Estimate how overfitting varies with number of filters.

In [38]:
def make_CNN(window_size, filter_length,  nb_filter=4, nb_input_series=1, nb_outputs=1):
    """
    window_size (int): number of observations in each input sequence
    filter length (int): length of the convolutional layer's filters
    nb_filter (int): number of filters learned in the convolutional layer
    nb_input_series (int): number of features of the input timeseries (1 for a univariate timeseries)
    nb_outputs (int): number of features being predicted (equal to nb_input_series 
        for predicting a timeseries at a set horizon)
    """
    model = Sequential((
        # The convolutional layer learns `nb_filter` filters (aka kernels), 
        # each of size `(filter_length, nb_input_series)`.  
        # Its output will have shape `(None, window_size - filter_length + 1, nb_filter)` ,  
        # i.e., for each position in the input timeseries, the activation of each filter at that position.
        Conv1D(filters=nb_filter, kernel_size=filter_length, activation='relu', input_shape=(window_size, nb_input_series)),
        Flatten(),
        Dense(nb_outputs, activation='linear'), # For classification, a 'sigmoid' activation function would be used
    ))
    model.compile(loss='mse', optimizer='adam', metrics=['mae'])
    
    return model

In [95]:
model1 = make_CNN(window_size=23, filter_length=5, nb_filter=1)
model4 = make_CNN(window_size=23, filter_length=5, nb_filter=4)
model8 = make_CNN(window_size=23, filter_length=5, nb_filter=8)
model16 = make_CNN(window_size=23, filter_length=5, nb_filter=16)

In [96]:
print('input shape:', model1.layers[0].input_shape)
print('output shape:', model1.layers[-1].output_shape)

input shape: (None, 23, 1)
output shape: (None, 1)


In [41]:
def make_timeseries_instances(timeseries, window_size):
    # Convert 1D vectors to 2D column vectors
    timeseries = np.atleast_2d(timeseries)
    if timeseries.shape[0] == 1:
        timeseries = timeseries.T 
    
    if not 0 < window_size < timeseries.shape[0]:
        raise ValueError('Please set 0 < window size < timeseries length')
    
    # `X `is the tensor containing the inputs for the model
    # each row of `X` is a sequence of `window_size` observations from the timeseries
    X = [timeseries[start:start + window_size] for start in range(0, timeseries.shape[0] - window_size)]
    
    # for training the model, the array's dimensions must match the input layer of the CNN
    # that is, a 3D array of shape (timeseries.shape[0] - window_size, window_size, nof_ts_variables)
    X = np.atleast_3d(np.array(X))
    
    # For each row of `X`, the corresponding row of `y` is the 
    # desired output -- in this case, the subsequent value in the timeseries 
    y = timeseries[window_size:]
    
    return X, y

In [93]:
x_data, y_data = make_timeseries_instances(dataSet,23)

test_ratio = 0.01 # In real life you'd usually want to use 0.2 - 0.5
test_size = int(test_ratio * len(x_data)) 

# the "most recent" values are used for testing the model to avoid look-ahead bias
X_train, X_test, y_train, y_test = x_data[:-test_size], x_data[-test_size:], y_data[:-test_size], y_data[-test_size:]

In [97]:
[i.shape for i in [X_train, X_test, y_train, y_test]]

[(19778, 23, 1), (199, 23, 1), (19778, 1), (199, 1)]

In [98]:
model1.fit(X_train, y_train, epochs=10, batch_size=2, validation_data=(X_test, y_test)) # clearly, a production run would have more data, and run for more epochs or until a call-back condition
model16.fit(X_train, y_train, epochs=10, batch_size=2, validation_data=(X_test, y_test)) # clearly, a production run would have more data, and run for more epochs or until a call-back condition
model4.fit(X_train, y_train, epochs=10, batch_size=2, validation_data=(X_test, y_test)) # clearly, a production run would have more data, and run for more epochs or until a call-back condition
model8.fit(X_train, y_train, epochs=10, batch_size=2, validation_data=(X_test, y_test)) # clearly, a production run would have more data, and run for more epochs or until a call-back condition

Epoch 1/10
9889/9889 [==============================] - 12s 1ms/step - loss: 0.0219 - mae: 0.0659 - val_loss: 0.0030 - val_mae: 0.0549
Epoch 2/10
9889/9889 [==============================] - 11s 1ms/step - loss: 0.0011 - mae: 0.0288 - val_loss: 0.0035 - val_mae: 0.0592
Epoch 3/10
9889/9889 [==============================] - 11s 1ms/step - loss: 0.0011 - mae: 0.0287 - val_loss: 0.0034 - val_mae: 0.0580
Epoch 4/10
9889/9889 [==============================] - 11s 1ms/step - loss: 0.0011 - mae: 0.0288 - val_loss: 0.0031 - val_mae: 0.0558
Epoch 5/10
9889/9889 [==============================] - 11s 1ms/step - loss: 0.0011 - mae: 0.0288 - val_loss: 0.0027 - val_mae: 0.0517
Epoch 6/10
9889/9889 [==============================] - 11s 1ms/step - loss: 0.0011 - mae: 0.0287 - val_loss: 0.0031 - val_mae: 0.0560
Epoch 7/10
9889/9889 [==============================] - 11s 1ms/step - loss: 0.0011 - mae: 0.0288 - val_loss: 0.0026 - val_mae: 0.0513
Epoch 8/10
9889/9889 [==============================] -

In [100]:
model16.summary()
model16.get_layer('conv1d_19').weights

Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_19 (Conv1D)           (None, 19, 16)            96        
_________________________________________________________________
flatten_19 (Flatten)         (None, 304)               0         
_________________________________________________________________
dense_19 (Dense)             (None, 1)                 305       
Total params: 401
Trainable params: 401
Non-trainable params: 0
_________________________________________________________________


[<tf.Variable 'conv1d_19/kernel:0' shape=(5, 1, 16) dtype=float32, numpy=
 array([[[-0.1554797 , -0.061258  ,  0.18947959, -0.20520496,
          -0.2838157 ,  0.17266205, -0.23266736, -0.12865904,
           0.16442642,  0.01958015,  0.20301566,  0.10959566,
          -0.10533132, -0.06308445, -0.07984362, -0.10609747]],
 
        [[ 0.25256202, -0.06881958, -0.12310731, -0.03293546,
          -0.15601605, -0.062581  ,  0.01405075,  0.07854968,
          -0.29065645, -0.00434351, -0.14412318, -0.05832983,
           0.14483806,  0.13173178, -0.10854668, -0.05814718]],
 
        [[ 0.06509399, -0.05361547, -0.23208347,  0.16958866,
           0.08947503,  0.10019264,  0.15304455,  0.20125335,
          -0.10137782,  0.1018483 , -0.16904674,  0.04577584,
           0.19503842, -0.25250164, -0.0723935 , -0.11415143]],
 
        [[-0.14714406, -0.25972706,  0.08877577, -0.10010441,
           0.22493145, -0.23983283,  0.13612503, -0.23696391,
           0.15701078, -0.25077957, -0.2302263

In [110]:
results={'model1':{},'model4':{},'model8':{},'model16':{}}
results['model1']['pred_train']=model1.predict(X_train, verbose=1)
results['model4']['pred_train']=model4.predict(X_train, verbose=1)
results['model8']['pred_train']=model8.predict(X_train, verbose=1)
results['model16']['pred_train']=model16.predict(X_train, verbose=1)
results['model1']['MSE_train'] = mean_squared_error(y_train, results['model1']['pred_train'])
results['model4']['MSE_train']=mean_squared_error(y_train, results['model4']['pred_train'])
results['model8']['MSE_train']=mean_squared_error(y_train, results['model8']['pred_train'])
results['model16']['MSE_train']=mean_squared_error(y_train, results['model16']['pred_train'])
results['model1']['pred_test']=model1.predict(X_test, verbose=1)
results['model4']['pred_test']=model4.predict(X_test, verbose=1)
results['model8']['pred_test']=model8.predict(X_test, verbose=1)
results['model16']['pred_test']=model16.predict(X_test, verbose=1)
results['model1']['MSE_test'] = mean_squared_error(y_test, results['model1']['pred_test'])
results['model4']['MSE_test']=mean_squared_error(y_test, results['model4']['pred_test'])
results['model8']['MSE_test']=mean_squared_error(y_test, results['model8']['pred_test'])
results['model16']['MSE_test']=mean_squared_error(y_test, results['model16']['pred_test'])
results['model1']['pred_train'].shape

7/7 [==============================] - 0s 2ms/step


(19778, 1)

In [111]:
for i in results:
  print(i,":\t MSE Train:\t ",results[i]['MSE_train'],"\t MSE Test:\t ",results[i]['MSE_test'])

model1 :	 MSE Train:	  0.001121276548615165 	 MSE Test:	  0.00339578959338725
model4 :	 MSE Train:	  0.00018982226420859017 	 MSE Test:	  0.00012080391847579508
model8 :	 MSE Train:	  7.44003040020648e-05 	 MSE Test:	  1.694703915340146e-05
model16 :	 MSE Train:	  5.196235975509257e-05 	 MSE Test:	  1.6995689525811264e-05


Which is reasonable, the more filters we apply, the more degrees of freedom we have to capture the behavior of the data. We are beginning to overfit somewhere between 8 filters and 16 filters, which is likely dependent on the choice of kernel size.
____
4.   Use L1 regularization to reduce the variance

Yeah, I'm tired, so I'm not going to regenerate and retrain all the models to do a ridge regression. How you would do this is you would pick the optimal model, and then add in a penalty equal to the L1 norm of the weights, and then retrain the models for a variety of penalty strengths. The unpenalized model ($\lambda$=0) is what we just trained, but given the sensibility of those results, I'm not sure I'm generating the models appropriately or taking enough data. You then pick the $\lambda$ that optimizes the regression, by whatever definition of "optimize" is relevant.
___
5.   Ljung-Box test for autocorrelations in the residuals

In [103]:
residuals= y_test-results['model16']['pred_test']

In [104]:
lb, p = sm.stats.diagnostic.acorr_ljungbox(residuals, lags=50, boxpierce=False)

In [105]:
p

array([4.02158711e-032, 3.48978164e-050, 1.16917621e-059, 5.60624670e-064,
       8.44926388e-066, 4.95251458e-066, 1.64174683e-065, 8.89735277e-065,
       3.73177311e-064, 1.58130531e-063, 5.03567774e-063, 1.20550688e-062,
       2.30496597e-062, 3.25372988e-062, 4.59430053e-062, 5.36950425e-062,
       8.36718815e-062, 2.41830100e-061, 1.02430811e-060, 4.10175342e-060,
       1.14286749e-059, 1.43061140e-059, 1.69802019e-060, 3.51020415e-063,
       1.88863239e-068, 3.53283503e-073, 1.58475398e-077, 1.48045740e-081,
       2.11743806e-085, 3.69229138e-088, 1.30144113e-089, 3.97615239e-090,
       2.86845638e-090, 1.73181713e-090, 9.57892741e-091, 2.26525961e-091,
       1.45422270e-092, 2.77775790e-094, 7.20817238e-097, 8.37064159e-100,
       8.99692481e-103, 4.12508025e-105, 3.34102548e-106, 2.60210126e-106,
       5.52274659e-106, 1.84626385e-105, 6.78734613e-105, 2.42178988e-104,
       7.62202877e-104, 2.11192038e-103])

So we do seem to be capturing all of the autoregressive features as these are all round-off errors.